In [2]:
from pylab import *
import numpy as np
%matplotlib inline
from matplotlib import rc
rc('figure',figsize=(10,5))
rc('font',size=20)
rc('text',usetex=True)
pgf_with_rc_fonts = {"pgf.texsystem": "pdflatex"}
matplotlib.rcParams.update(pgf_with_rc_fonts)
import pymc3

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [13]:
data = np.genfromtxt('SCPUnion2.1_mu_vs_z.txt')
zsn = data.T[1]
musn = data.T[2]
dmusn = data.T[3]



In [18]:
h = 0.7
omegal = 0.7
omegar = 4.3 * h**(-2) * 1e-5
omegam= 0.3 - omegar 

In [19]:
z = np.logspace(-1, 4, 1000)

In [23]:
omr = omegar * (1 + z)**4
omm = omegam * (1 + z)**3
oml= omegal + z*0
total = omr + omm + oml

In [25]:
plt.plot(omm,oml)

NameError: name 'omm' is not defined

In [ ]:
plot(z,om_l, 'b', label = r'$\Omega_\Lambda(z)$')
plot(z,om_r, 'k', label = r'$\Omega_R(z)$')
plot(z,om_m, 'g', label = r'$\Omega_{NR}(z)$')
plot(z,total, 'r', lw=2, label = 'Total')
legend(loc='upper left')
title('Densities as a function of Redshift')

In [17]:
plt.figure()
plt.plot(z,om_l,'b')
plt.plot
plt.xlim([0,2.0])
plt.xlabel('zs')
plt.ylabel('m_mod')
plt.grid(b=True,which='major')
plt.grid(b=True,which='minor')
plt.legend(loc='lower right')
plt.show()

NameError: name 'dz' is not defined

<Figure size 432x288 with 0 Axes>

In [9]:
cosmo_lcdm = {'h':0.7, 'omega_M_0':0.3, 'omega_lambda_0':0.7, 'w0':-1.}
cosmo_flat = {'h':0.7, 'omega_M_0':1, 'omega_lambda_0':0, 'w0':-1.}
cosmo_open = {'h':0.7, 'omega_M_0':0.3, 'omega_lambda_0':0, 'w0':-1.}

errorbar(zsn, musn, yerr=dmusn, fmt='ko')
zvalues = linspace(0,2,10000)
def propdist(z,cosmo,zres=0.001,accurate=False):
    ### z range for integration
    zmax=np.max(z)
    if zmax < zres:
        nb=101
    else:
        nb=zmax/zres+1
    zvals=np.linspace(0.,zmax,nb)
    ### integrate
    cumulative=np.zeros(nb)
    cumulative[1:]=scipy.integrate.cumtrapz(1./e_z(zvals,cosmo),zvals)
    ### interpolation to input z values
    propdist=np.interp(z,zvals,cumulative)
    ### curvature
    omega=cosmo['omega_M_0']+cosmo['omega_lambda_0']
    k=np.abs(1-omega)
    if omega == 1:
        propdist=propdist
    elif omega < 1:
        propdist=np.sinh(np.sqrt(k)*propdist)/np.sqrt(k)
    elif omega > 1:
        propdist=np.sin(np.sqrt(k)*propdist)/np.sqrt(k)
    ### returning
    return(propdist*2.99792458e5/100/cosmo['h'])
def lumdist(z,cosmo,zres=0.001,accurate=False):
    return(propdist(z,cosmo,zres=zres,accurate=accurate)*(1+z))
def musn1a(z, cosmo):
    
    dlum = lumdist(z, cosmo)*1e6
    return(5*np.log10(dlum)-5+5*np.log10(cosmo['h']/0.7))


plt.figure()
plt.plot(zvalues,musn1a(zvalues, cosmo_flat), lw=2, color='blue', 
     label='Flat matter only')
plt.plot(zvalues,musn1a(zvalues, cosmo_open), lw=2, color='green', 
     label='Open matter only')
plt.plot(zvalues, musn1a(zvalues, cosmo_lcdm), lw=2, color='red', 
     label='$\Lambda$CDM')
plt.xscale('log')
plt.xlim(1e-2,2)
plt.ylim(33,46)
plt.xlabel('Redshift')
plt.ylabel('Distance Modulus')
plt.legend(loc='upper left')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.
  


TypeError: 'numpy.float64' object cannot be interpreted as an index

Error in callback <function post_execute at 0x7fec26f95b90> (for post_execute):


OSError: [Errno 2] No such file or directory: 'latex'

OSError: [Errno 2] No such file or directory: 'latex'

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 0 Axes>

In [17]:
class Data():
    def __init__(self, xvals=None, yvals=None, errors = None, model=None, prior=False):
        self.prior = prior
        self.model = model
        self.xvals = xvals
        self.yvals = yvals
        if not self.prior:
            if np.size(np.shape(errors)) == 1:
                self.covar=np.zeros((np.size(errors),np.size(errors)))
                self.covar[np.arange(np.size(errors)),np.arange(np.size(errors))]=errors**2
            else:
                self.covar = errors
            self.invcov = np.linalg.inv(self.covar)
    
    def __call__(self,*pars):
        if  not self.prior:
            val=self.model(self.xvals,pars[0])
            chi2=np.dot(np.dot(self.yvals-val,self.invcov),self.yvals-val)
        else:
            chi2 = self.model(self.xvals, pars[0])
        return(-0.5*chi2)

In [19]:
def newmusn(x, pars):
    cosmo = {'h':pars[0], 'omega_M_0':pars[1], 
             'omega_lambda_0':pars[2], 'w0':pars[3]}
    val = musn1a(x, cosmo)
    val[isnan(val)] = -1e30
    return val
data = Data(xvals=zsn, yvals=musn, errors=dmusn, model=newmusn)


In [10]:
fid_params = {'h':0.7, 'om':0.3, 'ol':0.7, 'w':-1.}
def ll_model(datasets, variables = ['h','om', 'ol', 'w'], fidvalues = fid_params):
    if (isinstance(datasets, list) is False): datasets=[datasets]
    h  = pymc3.Uniform('h', 0., 1.2, value = fid_params['h'], observed = 'h' not in variables)
    om = pymc3.Uniform('om', 0,1.2, value = fid_params['om'], observed = 'om' not in variables)
    ol = pymc3.Uniform('ol', 0,1.2, value = fid_params['ol'], observed = 'ol' not in variables)
    w = pymc3.Uniform('w', -2,0, value = fid_params['w'], observed = 'w' not in variables)
    @pymc.stochastic(trace=True,observed=True,plot=False)
    def loglikelihood(value=0, h=h, om=om,ol=ol,w=w):
        ll=0.
        pars = np.array([h, om,ol,w])
        for ds in datasets:
            ll=ll+ds(pars)
        return(ll)
    return(locals())
def run_mcmc(data,niter=80000, nburn=20000, nthin=1, variables=['h', 'om', 'ol', 'w'], external=None, ll_model=ll_model,delay=1000):
    chain = pymc3.MCMC(ll_model(data, variables, fidvalues=fid_params))
    chain.use_step_method(pymc3.AdaptiveMetropolis,chain.stochastics,delay=delay)
    chain.sample(iter=niter,burn=nburn,thin=nthin)
    ch ={}
    for v in variables: ch[v] = chain.trace(v)[:]
    return ch


In [11]:
with disaster_model:
    trace = pm.sample(10000)




NameError: name 'disaster_model' is not defined